## Introduction:

The Naive Bayes Classifier can be applied to the financial data regarding a publicly traded company. The idea is to create a model that can assist in deciding whether to purchase a stock before earnings are released before market open the following day. What this classifier helps us determine is the probability that Stock A will increase by y%, given that a certain amount of events x have already occured. For example, assume that Stock A releases their earnings the morning after market open and at 3:55 PM an investor needs to make a decision on whether to buy or not buy. The investor could use this model to see what
scenario is most probable. The question that the model will answer is:

"What is the probability that Chegg's stock price will increase by at least 10%, at least once, before 12 pm of the following trading day, given that events x1 through x6 occur:

- x1 = Earnings report is released after the market closes
        -BMO
        -AMC
- x1 = Adjusted Stock Price is larger than when the previous earnings report was released 
- x2 = Current earnings per share (EPS) is greater than the EPS from the previous earnings report
- x3 = EPS Surprise percentage is positive
- x6 = Intraday return of the stock, on the day earnings are released, is greater than or equal to the median intraday return of the last 5 trading days
- x5 = Intraday return of the SPY etf is  





## Stating The Hypothesis

Null Hypothesis:

The percentage return of Stock, one full trading day after earnings are released, will be at least 5%, given that events x1 through x6 occured

Alternative Hypothesis:

The percentage return of Stock, one full trading day after earnings are released, will NOT be at least 5%, given that events x1 through x6 occured


### Explaining Events

The first event is necessary for this analysis to occur because the analysis is assuming that a earnings report has been released after market open.

The first event could be telling us that the market thinks that the company is going to perform well in the upcoming earnings report, and it reflects that through the price. The market capitalization was used because it takes into account stock-splits. Moreover, the first event could also signal that the market is overvaluing the future performance of the company, potentially making it a relatively bad time to buy the stock. 

The second event informs the investor that that growth is currently in place or that the company is becoming more efficient by decreasing their cost of goods/expenses. Both could lead to an increase interest in the company and drive the stock price up based on demand.   

The third event gives us a better understanding of how incorrect the market is at estimating the performance of a company. The higher the percentage error, also known as the surprise percentage, the better, because it signals under-valuation. During these periods, the investor has the opportunity to purchase a stock for the low and then sell at a 'fair' price.

The fourth event tells investors that the company has a relatively bright financial future. It is common to see the stock price of a company increase even after a poor earnings report because the guidance given by the company leaders was positive. At the end of the day, investing is always forward looking and people are willing to pay a premium for that.

The fifth event informs the investor on how the overall market is performing in the span of one week. The S&P 500 is index that tracks slightly over 500 publicly traded companies in the United States. This index is used as a benchmark to gage how the overall market is performing. If by 3:00 PM, the return on the S&P 500 was higher than that of last week, it could potentially signal that the market has a positive outlook or that a recorrection is taking place.

The sixth event shows that the market has a positive sentiment regarding earnings. It is common to see a stock's intraday return fluctulate throughout the week and it is possible that one of those days, the market overvalued the stock and increased the average intraday return. Thus, the median was used to  better reflect the distribution of the most recent intraday returns. 

## Gathering Financial Data

The majority of the financial data will be pulled from alphavantage's financial APIs. The first point of interest is to know how the alphavantage stores information regarding what publicly traded companies are releasing earnings in a given time period.





#######
Explain the thought process of how to go about getting the data to prove your hypothesis. Explain that you will be using the naive bayes that deals with binary values for the 'independent' variables. Explain the Yahoo Finance API and how to pull data from it. Explain the prunning process and how the model can be easily tested. Explain why its called Naive Bayes (Assumption that the x variables are independent of each other is naive to think because that rarely happpens in real life. The events listed above could potentially be corrrelated to one another -  this will be analyed in the descriptive section using scatter plots. Ideally, the benchmark probability should be higher than a coin toss, meaning that the model should be able to predict this null hypothesis at least 51% of the time. Knowing this probability is useful because a simulation can be dervived from it (Monte Carlo)

In [264]:
#create a function that:
#pulls the upcoming earnings data from alphavantage
#individual earnings table for each stock in the upcoming earnings table
#daily adjusted prices

#inputs include:
#type of data to retrieve
#api key

def alphaDataRetriever(api_key, data_of_interest, symbol,date_of_interest):
    #import necessary libraries
    import pandas as pd
    import csv
    import requests
    
    #remember that alphavantage only takes 5 calls per mintute and 500 throughout the entire day
    #for the earnings calendar call, symbol is not important
    
    if data_of_interest == 'earnings_calendar':
        
        CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=' + api_key

        with requests.Session() as s:

            download = s.get(CSV_URL)
            decoded_content = download.content.decode('utf-8')
            cr = csv.reader(decoded_content.splitlines(), delimiter=',')
            earnings_calendar = pd.DataFrame(data =cr)

        #edit the dataframe:
        #make the first row the column names
        earnings_calendar = earnings_calendar.rename(columns=earnings_calendar.iloc[0])

        #delete the first row since it has been used already to name the columns
        earnings_calendar = earnings_calendar.iloc[1:,:]

        #filter by the date of interest and loc the dataframe
        ec = earnings_calendar.loc[earnings_calendar.reportDate == date_of_interest]
        
        #not all stocks have an EPS estimate, these should be excluded from the dataframe
        #dataframe = dataframe[dataframe.column then the condition]
        ec = ec[ec.estimate != '']

        #not all stocks are traded in USD, but this project will only focus on the USD currency
        ec = ec[ec.currency == 'USD']

        #remove the irrelevant columns
        ec = ec.drop(['fiscalDateEnding','currency'],1)

        #display dataframe
        return ec
        
        
    elif data_of_interest == 'historical_earnings':

        #format of how the url should be structured before using the api
        earnings_history_url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=' + symbol +'&apikey=' + api_key

        #makes the request using the formatted url
        r = requests.get(earnings_history_url)
        data = r.json()

        #the data variable stores the data in json structure and the value of key = quarterlyEarnings is the point of interest
        quarterly_earnings = data['quarterlyEarnings']

        #convert this json into a dataframe
        quarterly_earnings = pd.DataFrame(quarterly_earnings)
        
        quarterly_earnings = quarterly_earnings.drop('fiscalDateEnding',1)
        
        return quarterly_earnings
    
    
    else:
        #getting the time series data
        #this url contains the format needed to make the api call for time series data
        daily_prices_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=' + symbol + '&outputsize=full' +'&apikey=' + api_key

        #makes the api request/call
        daily_price_request = requests.get(daily_prices_url)

        #stores the values into a dictionary
        daily_prices = daily_price_request.json()
        
        #stores the relevant information into a dictionary
        dictionary_of_dp = daily_prices['Time Series (Daily)']

        #convert this json into a dataframe, where the keys are the rows and the values are the columns 
        historical_prices = pd.DataFrame.from_dict(dictionary_of_dp, orient = 'index')
        
        #turn column values into numeric values
        historical_prices = historical_prices.apply(pd.to_numeric)
    
        #perform return calculations for each specific date
        historical_prices['Intraday_Returns'] = round((historical_prices['5. adjusted close']/historical_prices['1. open']-1)*100,4)

        #find the previous 5 day median intraday return and create the column 
        #the value will take into account the previous 5 trading days
        historical_prices['Weekly_Median_Return'] = historical_prices['Intraday_Returns'].rolling(window=5).median().shift(-5)

        #the indexes are the dates thus make that a column labeled reportedDate because that will be the key column for the inner join
        historical_prices['trading_date'] = historical_prices.index
        
        #reset the index since its a column now
        historical_prices.reset_index(drop=True, inplace=True)
        
        #remember that the model could break if there is not enough historical data
        return historical_prices
        

In [323]:
api_key = 'TQ6TR98HVUPE3L9Z'
data_of_interest = 'earnings_calendar'
symbol = 0
date_of_interest = '2021-10-07'

alphaEarningsCalendar  = alphaDataRetriever(api_key,data_of_interest,symbol,date_of_interest)
alphaEarningsCalendar

,symbol,name,reportDate,estimate
41,ACCD,Accolade Inc,2021-10-07,-0.56
771,CAG,Conagra Brands Inc,2021-10-07,0.48
2086,HELE,Helen of Troy Ltd,2021-10-07,2.16
2723,LW,Lamb Weston Holdings Inc,2021-10-07,0.38
4410,TLRY,Tilray Inc - Class 2,2021-10-07,-0.07


In [326]:
api_key = 'TQ6TR98HVUPE3L9Z'
data_of_interest = 'historical_earnings'
symbol = 'TLRY'
date_of_interest = '2021-10-07'

alphaHistoricalEarnings  = alphaDataRetriever(api_key,data_of_interest,symbol,date_of_interest)
alphaHistoricalEarnings.head()

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-07-28,0.4407,-0.114,0.5547,486.5789
1,2021-04-12,-1.75,-0.08,-1.67,-2087.5
2,2021-02-17,-0.0433,-0.1333,0.09,67.5169
3,2020-11-09,-0.1592,-0.2038,0.0446,21.8842
4,2020-08-10,-0.179,-0.4043,0.2253,55.7259


In [327]:
api_key = 'TQ6TR98HVUPE3L9Z'
data_of_interest = 'daily_adjusted'
symbol = 'TLRY'
date_of_interest = '2021-10-07'

alphaDailyAdjusted  = alphaDataRetriever(api_key,data_of_interest,symbol,date_of_interest)
alphaDailyAdjusted.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,trading_date
0,10.75,11.34,10.6807,11.10,11.10,18162484,0.0,1.0,3.2558,-2.5408,2021-10-05
1,11.02,11.11,10.6800,10.74,10.74,14980747,0.0,1.0,-2.5408,-1.2400,2021-10-04
2,11.29,11.37,10.8400,11.15,11.15,18180296,0.0,1.0,-1.2400,-1.5190,2021-10-01
3,11.17,11.53,10.9500,11.29,11.29,20291033,0.0,1.0,1.0743,-1.5190,2021-09-30
4,11.73,11.77,11.0900,11.11,11.11,17441811,0.0,1.0,-5.2856,-1.4754,2021-09-29


In [328]:
api_key = 'TQ6TR98HVUPE3L9Z'
data_of_interest = 'daily_adjusted'
symbol = 'SPY'
date_of_interest = 0

alphaSPY  = alphaDataRetriever(api_key,data_of_interest,symbol,date_of_interest)
alphaSPY.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,trading_date
0,430.24,435.4900,429.39,433.10,433.10,90682520,0.0,1.0,0.6647,-1.0069,2021-10-05
1,433.00,433.9600,426.36,428.64,428.64,128569957,0.0,1.0,-1.0069,-0.1700,2021-10-04
2,430.98,436.0329,427.23,434.24,434.24,129240106,0.0,1.0,0.7564,-0.1700,2021-10-01
3,436.02,436.7700,428.78,429.14,429.14,140505979,0.0,1.0,-1.5779,-0.0384,2021-09-30
4,435.19,437.0400,433.85,434.45,434.45,82329213,0.0,1.0,-0.1700,0.4151,2021-09-29


In [270]:
#create a function that takes the dataframes and merges AND creates x variables of the model
#this function will create the training and testing data

def variableTransformer(historical_earnings,upcoming_earnings, daily_adjusted,spy,output_Type):
    
    #import necessary libraries
    import pandas as pd

    #don't alter the dataframes that were used as inputs, instead, make a copy of each 
    #this will be used to get the class variable of whether the stock price increased by 5% the day after earnings are released
    #timing of the earnings release is not relevant for analysis
    
    
    
    ######################################################################################
    
    #this variable hold alls the daily adjusted values, but this will be altered/edited, whereas the master_daily will be referenced 
    historical_prices = daily_adjusted.copy(deep=True)

    #this variable holds a stock's individual earnings history
    earnings_hist = historical_earnings.copy(deep=True)
    
    #for training purposes,only keep the rows that are relevant.
    #the api feeds in the most current earnings and as a result, the historical prices will not line up
    #let those rows be used for the testing portion, while the remaining events get analyzed

    #create a list of all the reportedDates in the earnings dataframe
    reported_dates_list  = earnings_hist.reportedDate.tolist()

    #remove the most recent reporting date
    reported_dates_list = reported_dates_list[1:]

    #code below is used to calculate the Intraday Return Event for stock A and SPY as well as the adjusted price event:
    
    #this line filters out all the dates that are not a previous reporting date
    historical_prices = historical_prices.loc[historical_prices['trading_date'].isin(reported_dates_list)]

    #add the column that determines if the intraday return of the earnings release date is larger than that of the median
    historical_prices['intraday_event'] = (historical_prices['Intraday_Returns'] > historical_prices['Weekly_Median_Return']).astype(int)

    #add the column that determines if the adjusted share price is larger than that of last earnings call
    historical_prices['adjusted_price_event'] = (historical_prices['5. adjusted close'] > historical_prices['5. adjusted close'].shift(-1)).astype(int)

    #add the column that determines if the intraday return of the earnings release date is larger than that of the median
    historical_prices['spy_event'] = (spy['Intraday_Returns'] > spy['Weekly_Median_Return']).astype(int)

    #before joining, there needs to be a common column name
    #change the name of the date columns to column_of_interest
    earnings_hist.rename(columns = {'reportedDate':'column_of_interest'}, inplace = True)
    historical_prices.rename(columns = {'trading_date':'column_of_interest'}, inplace = True)
    spy.rename(columns = {'trading_date':'column_of_interest'}, inplace = True)
    
    #remove the first row of the earnings_hist table because that is upcoming earnings report and it won't help in training the model 
    #it can be used for testing only
    earnings_hist = earnings_hist.iloc[1:,:]
    
    #historical_prices dataframe editting:
    #in historical_prices, change the column name of '5. adjusted close' to 'stock_adjusted_close'
    historical_prices.rename(columns = {'5. adjusted close':'stock_adjusted_close'}, inplace = True)

    #in historical_prices, change the column name of 'Intraday_Returns' to 'stock_intraday_returns'
    historical_prices.rename(columns = {'Intraday_Returns':'stock_intraday_returns'}, inplace = True)
    
    #in historical_prices, change the column name of 'Weekly_Median_Return' to 'stock_weekly_median_return'
    historical_prices.rename(columns = {'Weekly_Median_Return':'stock_weekly_median_return'}, inplace = True)

    #spy dataframe editting:
    #in spy, change the column name '5. adjusted close' to 'stock_adjusted_close'
    spy.rename(columns = {'5. adjusted close':'spy_adjusted_close'}, inplace = True)

    #in spy, change the column name 'Intraday_Returns' to 'stock_adjusted_close'
    spy.rename(columns = {'Intraday_Returns':'spy_intraday_returns'}, inplace = True)
    
    #in historical_prices, change the column name of 'Weekly_Median_Return' to 'spy_weekly_median_return'
    spy.rename(columns = {'Weekly_Median_Return':'spy_weekly_median_return'}, inplace = True)

    #keep the relevant columns in the historical_prices df only
    columns_to_keep = ['stock_adjusted_close','stock_intraday_returns','stock_weekly_median_return','column_of_interest', 'intraday_event','adjusted_price_event','spy_event']
    historical_prices = historical_prices[columns_to_keep]
    
    #join the dataframes so that earnings calendar is on the left     
    inner_merged_master = earnings_hist.merge(historical_prices,on='column_of_interest').merge(spy,on='column_of_interest')
    
    #convert the following columns to numeric
    inner_merged_master["reportedEPS"] = pd.to_numeric(inner_merged_master["reportedEPS"])
    inner_merged_master["estimatedEPS"] = pd.to_numeric(inner_merged_master["estimatedEPS"])
    inner_merged_master["surprisePercentage"] = pd.to_numeric(inner_merged_master["surprisePercentage"])
    
    #adding the last two events:
    #add the column that determines if current earnings were greater than last time
    inner_merged_master['earnings_event'] = (inner_merged_master['reportedEPS'] > inner_merged_master['reportedEPS'].shift(-1)).astype(int)

    # hist.loc[:,'DependentVariable'] = (hist['Close'] > hist['Close'].shift(-1)).astype(int)

    #add the column that determines if the surprise variable is positive or not
    inner_merged_master['surprise_event'] = (inner_merged_master['surprisePercentage'] > 0).astype(int)
    
    #only keep the columns that are relevant to the analysis
    
    final_columns_to_keep = ['column_of_interest','reportedEPS','estimatedEPS','surprise','surprisePercentage','stock_adjusted_close','stock_intraday_returns','stock_weekly_median_return',
                            'spy_adjusted_close','spy_intraday_returns','spy_weekly_median_return','earnings_event','surprise_event','adjusted_price_event','intraday_event','spy_event']
    
    inner_merged_master = inner_merged_master[final_columns_to_keep]
    
    #add the class variable:
    
    ######################################################################################
    
    #this variable holds all the daily adjusted values for a particular stock, not just values that pertain to date of interest
    master_daily_adjusted = daily_adjusted.copy(deep=True)
    
    
    #master_hist is the dataframe that only includes the dates of that match up with the dates that earnings are reported
    master_hist = master_daily_adjusted.loc[master_daily_adjusted['trading_date'].isin(reported_dates_list)]
        
    #store the index values from the list. This will tell us the index where the earnings reported date is located at
    row_index_list = master_hist.index.tolist()


    #alter the values of the elements in the list so that element = element -1
    #this will give us the value of the index pertaining to returns one trading day following reported release
    #the way the dataframe is filtered is by most recent date to the most future date
    
    for i in range( len(row_index_list)):
        row_index_list[i] = row_index_list[i] - 1

    
    #lock the dataframe so that only the row indexes from the list come up
    master_daily_adjusted = master_daily_adjusted.loc[row_index_list,:]

    #rename the column so that the analyst knows what this value is. It's the price of the stock one full trading day after earnings have been released
    master_daily_adjusted.rename(columns = {'5. adjusted close':'after_report_adjusted_price'},inplace=True)
    

    #up to this point, the after_report_adjusted_price column exists
    #store the values of the after_report_adjusted_price in a list to 
    reported_adjusted_close_prices = master_daily_adjusted['after_report_adjusted_price'].tolist()
    
    
    #add the values from the list to a new column in the inner master df
    inner_merged_master['after_report_adjusted_price'] = reported_adjusted_close_prices
        
    #calculate the adjusted price in a new column
    inner_merged_master['return_pct_difference'] = round((inner_merged_master['after_report_adjusted_price']/inner_merged_master['stock_adjusted_close']-1)*100,2)

    #calculate the class variable
    inner_merged_master['class_variable'] = (inner_merged_master['return_pct_difference'] > 5).astype(int)
    
        
    #add an if statement that returns the full master dataset if modelprep is equal to false; this for the analyst to see how the numbers were derived
    #else return only the clean variables and the class variable;
    
    if output_Type == 'Logic':
        #return the dataframe in a way that shows the logic of how the model variables came to be
    
        inner_merged_master = inner_merged_master.reindex(columns=['column_of_interest', 'reportedEPS', 'estimatedEPS', 
                                                               'surprise','surprisePercentage','surprise_event', 'earnings_event',                                                                                                                             
                                                               'stock_adjusted_close','after_report_adjusted_price','return_pct_difference',
                                                               'class_variable','adjusted_price_event','stock_intraday_returns','stock_weekly_median_return',
                                                               'intraday_event','spy_adjusted_close','spy_intraday_returns',
                                                               'spy_weekly_median_return','spy_event'                                                              
                                                              ])
    
        return inner_merged_master
    
    elif output_Type == 'Clean_Variables':
        #return only the clean variables that will be used for analysis
        model_variables = ['surprise_event','earnings_event','adjusted_price_event','intraday_event','spy_event','class_variable']
        return inner_merged_master.loc[:,model_variables]
    
    
    else:
        #returns the master table
        return inner_merged_master

In [329]:
testing1 =  variableTransformer(alphaHistoricalEarnings,alphaEarningsCalendar,alphaDailyAdjusted,alphaSPY,output_Type='Clean_Variables')
testing1.head()

,surprise_event,earnings_event,adjusted_price_event,intraday_event,spy_event,class_variable
0,0,0,0,0,1,1
1,1,1,1,0,1,0
2,1,1,1,0,0,0
3,1,1,0,1,0,0
4,0,0,0,1,1,0


In [330]:
#break the data into x and y variables
#for training purposes the test data will consist of full year prediction
#the dataframe has the most recent quarters at the beginning, thus use the first 4 rows as the testing data



#the train variables will be the actual results of one year prior
x_train_variables = testing1.iloc[4:,0:5]
y_train_variables = testing1.iloc[4:,5]

#test variables will be the most recent year
x_test_variables = testing1.iloc[0:4,0:5]
y_test_variables = testing1.iloc[0:4,5]

print(x_train_variables.shape)
x_train_variables.head()

(8, 5)


,surprise_event,earnings_event,adjusted_price_event,intraday_event,spy_event
4,0,0,0,1,1
5,0,0,0,1,1
6,0,0,0,0,1
7,0,0,0,1,1
8,0,1,0,1,1


In [331]:
print(x_test_variables.shape)
x_test_variables

(4, 5)


,surprise_event,earnings_event,adjusted_price_event,intraday_event,spy_event
0,0,0,0,0,1
1,1,1,1,0,1
2,1,1,1,0,0
3,1,1,0,1,0


In [332]:
print(y_train_variables.shape)
y_train_variables.head()

(8,)


4    0
5    0
6    0
7    0
8    0
Name: class_variable, dtype: int32

In [333]:
print(y_test_variables.shape)
y_test_variables

(4,)


0    1
1    0
2    0
3    0
Name: class_variable, dtype: int32

In [ ]:
#descriptive section 

graph_values = [amount_of_unique_ogtweets,amount_of_unique_retweets]
tweet_label = ['Tweet','Retweet']

plt.bar(tweet_label,graph_values)    
plt.title('Tweets vs Retweets')
plt.xlabel('Tweet Type')
plt.ylabel('Tweet Frequency')
plt.show()

import matplotlib.pyplot as plt


In [334]:
from sklearn.naive_bayes import BernoulliNB
naive_bayes_model = BernoulliNB(binarize=0.0)
naive_bayes_model.fit(x_train_variables, y_train_variables)
print('The model predicted the Y Test variable correctly on %d percent of the Test data.'%(naive_bayes_model.score(x_test_variables,y_test_variables)*100))

The model predicted the Y Test variable correctly on 50 percent of the Test data.


In [309]:
print(naive_bayes_model.predict(x_test_variables))


[0 0 0 0]


In [300]:
y_test_variables

,class_variable
0,0
1,0
2,0
3,0


In [102]:
#api key = TQ6TR98HVUPE3L9Z
import pandas as pd
import csv
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey=TQ6TR98HVUPE3L9Z'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    earnings_calendar = pd.DataFrame(data =cr)
#     print(newDf)

earnings_calendar.head()    


,0,1,2,3,4,5
0,symbol,name,reportDate,fiscalDateEnding,estimate,currency
1,A,Agilent Technologies Inc,2021-11-22,2021-10-31,1.17,USD
2,AA,Alcoa Corp,2021-10-14,2021-09-30,1.71,USD
3,AACG,ATA Creativity Global,2021-11-10,2021-09-30,-0.15,USD
4,AADI,Aadi Bioscience Inc,2021-11-08,2021-09-30,-6,USD


In [24]:
#edit the dataframe:

#make the first row the column names
earnings_calendar = earnings_calendar.rename(columns=earnings_calendar.iloc[0])

#delete the first row since it has been used already to name the columns
earnings_calendar = earnings_calendar.iloc[1:,:]

#display head
earnings_calendar.head()

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
1,A,Agilent Technologies Inc,2021-11-22,2021-10-31,1.17,USD
2,AA,Alcoa Corp,2021-10-14,2021-09-30,1.71,USD
3,AACG,ATA Creativity Global,2021-11-10,2021-09-30,-0.15,USD
4,AADI,Aadi Bioscience Inc,2021-11-08,2021-09-30,-6,USD
5,AAIC,Arlington Asset Investment Corp - Class A,2021-11-01,2021-09-30,0.05,USD


In [25]:
#filter smallest to largest or in this case, most recent date
earnings_calendar = earnings_calendar.sort_values(by=['reportDate'])
earnings_calendar.head()

,symbol,name,reportDate,fiscalDateEnding,estimate,currency
3144,NOBDF,NOBDF,2021-10-05,2021-08-31,,USD
1601,EXFO,EXFO Inc,2021-10-05,2021-08-31,,USD
3922,SAR,Saratoga Investment Corp,2021-10-05,2021-08-31,0.5,USD
1820,FTGFF,FTGFF,2021-10-05,2021-08-31,,USD
3464,PEP,PepsiCo Inc,2021-10-05,2021-09-30,1.73,USD


In [26]:
#not all stocks have an estimate, these should be excluded from the dataframe
#dataframe = dataframe[dataframe.column then the condition]
earnings_calendar = earnings_calendar[earnings_calendar.estimate != '']

#not all stocks are traded in USD, but this project will only focus on the USD currency
earnings_calendar = earnings_calendar[earnings_calendar.currency == 'USD']

earnings_calendar.head()




,symbol,name,reportDate,fiscalDateEnding,estimate,currency
3922,SAR,Saratoga Investment Corp,2021-10-05,2021-08-31,0.5,USD
3464,PEP,PepsiCo Inc,2021-10-05,2021-09-30,1.73,USD
4419,TMQ,Trilogy Metals Inc,2021-10-05,2021-08-31,-0.02,USD
2673,LOOP,Loop Industries Inc,2021-10-05,2021-08-31,-0.25,USD
490,AYI,Acuity Brands Inc,2021-10-06,2021-08-31,2.87,USD


In [27]:
#run once
#remove the irrelevant columns
earnings_calendar = earnings_calendar.drop(['fiscalDateEnding','currency'],1)


In [30]:
#filter by the date of interest
date_of_interest = '2021-10-06'

#loc the dataframe
ec = earnings_calendar.loc[earnings_calendar.reportDate == date_of_interest]

#display
ec

#unfortunately, this api does not tell us when the earnings call is and that is very important to the analysis


,symbol,name,reportDate,estimate
490,AYI,Acuity Brands Inc,2021-10-06,2.87
4239,STZ,Constellation Brands Inc - Class A,2021-10-06,2.77
3856,RPM,RPM International Inc,2021-10-06,1.03
4474,TRNO,Terreno Realty Corp,2021-10-06,0.24
2615,LEVI,Levi Strauss & Co. Cls A,2021-10-06,0.37
3788,RGP,Resources Connection Inc,2021-10-06,0.27


In [32]:
#this can be added at the end in a different function

#get the data needed
import finnhub
finnhub_client = finnhub.Client(api_key="c5d5iiqad3i9ue38pn9g")

finhub_json = finnhub_client.earnings_calendar(_from=date_of_interest, to=date_of_interest, symbol="", international=False)


fin_e_calendar = finhub_json['earningsCalendar']

fin_e_calendar = pd.DataFrame(fin_e_calendar)
print(fin_e_calendar.shape)
fin_e_calendar.head()



(7, 9)


,date,epsActual,epsEstimate,hour,quarter,revenueActual,revenueEstimate,symbol,year
0,2021-10-06,None,0.377767,amc,3,None,1.483299e+09,LEVI,2021
1,2021-10-06,None,2.896306,bmo,4,None,9.800332e+08,AYI,2021
2,2021-10-06,None,0.272700,amc,1,None,1.786400e+08,RGP,2022
3,2021-10-06,None,1.037543,bmo,1,None,1.664228e+09,RPM,2022
4,2021-10-06,None,2.798518,bmo,2,None,2.336153e+09,STZ,2022


In [33]:
#only keep the relevant columns
fin_e_calendar = fin_e_calendar.loc[:,['symbol','hour','date']]
fin_e_calendar

,symbol,hour,date
0,LEVI,amc,2021-10-06
1,AYI,bmo,2021-10-06
2,RGP,amc,2021-10-06
3,RPM,bmo,2021-10-06
4,STZ,bmo,2021-10-06
5,IDT,amc,2021-10-06
6,RELL,amc,2021-10-06


In [34]:

#merge the dataframes on the symbol column
#make a copy to mess around with

copy1 = ec.copy()
copy2 = fin_e_calendar.copy()
# copy1

updatedDf = pd.merge(copy1, copy2,
                        how="left", on=["symbol"])
print(updatedDf.shape)
updatedDf.head()

(6, 6)


,symbol,name,reportDate,estimate,hour,date
0,AYI,Acuity Brands Inc,2021-10-06,2.87,bmo,2021-10-06
1,STZ,Constellation Brands Inc - Class A,2021-10-06,2.77,bmo,2021-10-06
2,RPM,RPM International Inc,2021-10-06,1.03,bmo,2021-10-06
3,TRNO,Terreno Realty Corp,2021-10-06,0.24,NaN,NaN
4,LEVI,Levi Strauss & Co. Cls A,2021-10-06,0.37,amc,2021-10-06


In [35]:
#remove the rows/stocks that dont have an earnings call time
updatedDf = updatedDf[updatedDf['hour'].notna()]

#keep the rows where hour = bmo for future analysis
updatedDf

,symbol,name,reportDate,estimate,hour,date
0,AYI,Acuity Brands Inc,2021-10-06,2.87,bmo,2021-10-06
1,STZ,Constellation Brands Inc - Class A,2021-10-06,2.77,bmo,2021-10-06
2,RPM,RPM International Inc,2021-10-06,1.03,bmo,2021-10-06
4,LEVI,Levi Strauss & Co. Cls A,2021-10-06,0.37,amc,2021-10-06
5,RGP,Resources Connection Inc,2021-10-06,0.27,amc,2021-10-06


In [36]:
#used to test the code in the section below
test = updatedDf.copy()
test

,symbol,name,reportDate,estimate,hour,date
0,AYI,Acuity Brands Inc,2021-10-06,2.87,bmo,2021-10-06
1,STZ,Constellation Brands Inc - Class A,2021-10-06,2.77,bmo,2021-10-06
2,RPM,RPM International Inc,2021-10-06,1.03,bmo,2021-10-06
4,LEVI,Levi Strauss & Co. Cls A,2021-10-06,0.37,amc,2021-10-06
5,RGP,Resources Connection Inc,2021-10-06,0.27,amc,2021-10-06


In [41]:
#take it one stock at a time
#What happens after you get the earnings calendar?

symbol = 'STZ'
apikey='TQ6TR98HVUPE3L9Z'

#format of how the url should be structured before using the api
earnings_history_url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=' + symbol +'&apikey=' + apikey

#makes the request using the formatted url
r = requests.get(earnings_history_url)
data = r.json()

#the data variable stores the data in json structure and the value of key = quarterlyEarnings is the point of interest
quarterly_earnings = data['quarterlyEarnings']

#convert this json into a dataframe
quarterly_earnings = pd.DataFrame(quarterly_earnings)
quarterly_earnings.head()




,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage
0,2021-05-31,2021-06-30,2.33,2.3491,-0.0191,-0.8131
1,2021-02-28,2021-04-08,1.82,1.5484,0.2716,17.5407
2,2020-11-30,2021-01-07,3.09,2.4095,0.6805,28.2424
3,2020-08-31,2020-10-01,2.76,2.5132,0.2468,9.8201
4,2020-05-31,2020-07-01,2.3,2.0453,0.2547,12.4529


In [42]:
quarterly_earnings.shape

(101, 6)

In [43]:
#getting the time series data

#remember that alphavantage only takes 5 calls per mintute and 500 throughout the entire day
symbol = 'STZ'

apikey='TQ6TR98HVUPE3L9Z'

#this url contains the format needed to make the api call for time series data

daily_prices_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=' + symbol + '&outputsize=full' +'&apikey=' + apikey

#makes the api request/call
daily_price_request = requests.get(daily_prices_url)

#stores the values into a dictionary
daily_prices = daily_price_request.json()

#make a copy
daily_prices_copy = daily_prices.copy()

print(daily_prices_copy)


{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'STZ', '3. Last Refreshed': '2021-10-04', '4. Output Size': 'Full size', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2021-10-04': {'1. open': '214.0', '2. high': '215.56', '3. low': '212.4', '4. close': '213.02', '5. adjusted close': '213.02', '6. volume': '1196629', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2021-10-01': {'1. open': '211.59', '2. high': '216.14', '3. low': '210.94', '4. close': '213.48', '5. adjusted close': '213.48', '6. volume': '1314205', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2021-09-30': {'1. open': '214.89', '2. high': '215.79', '3. low': '210.46', '4. close': '210.69', '5. adjusted close': '210.69', '6. volume': '1181815', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2021-09-29': {'1. open': '211.27', '2. high': '213.11', '3. low': '210.5', '4. close': '211.76', '5. adjusted close': '2

In [44]:
#stores the relevant information into a dictionary
dictionary_of_dp = daily_prices_copy['Time Series (Daily)']

#converts the dictionary into a list so that the index can be found to calculate the median
keys = list(dictionary_of_dp.keys())

print('There are %i trading days in the list' %len(keys))

There are 5517 trading days in the list


In [45]:
#display the json structure
dictionary_of_dp

{'2021-10-04': {'1. open': '214.0',
  '2. high': '215.56',
  '3. low': '212.4',
  '4. close': '213.02',
  '5. adjusted close': '213.02',
  '6. volume': '1196629',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2021-10-01': {'1. open': '211.59',
  '2. high': '216.14',
  '3. low': '210.94',
  '4. close': '213.48',
  '5. adjusted close': '213.48',
  '6. volume': '1314205',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2021-09-30': {'1. open': '214.89',
  '2. high': '215.79',
  '3. low': '210.46',
  '4. close': '210.69',
  '5. adjusted close': '210.69',
  '6. volume': '1181815',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2021-09-29': {'1. open': '211.27',
  '2. high': '213.11',
  '3. low': '210.5',
  '4. close': '211.76',
  '5. adjusted close': '211.76',
  '6. volume': '1422790',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2021-09-28': {'1. open': '211.67',
  '2. high': '212.03',
  '3. low':

In [46]:
#convert this json into a dataframe
historical_prices = pd.DataFrame.from_dict(dictionary_of_dp, orient = 'index')
historical_prices.head()


,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
2021-10-04,214.0,215.56,212.4,213.02,213.02,1196629,0.0000,1.0
2021-10-01,211.59,216.14,210.94,213.48,213.48,1314205,0.0000,1.0
2021-09-30,214.89,215.79,210.46,210.69,210.69,1181815,0.0000,1.0
2021-09-29,211.27,213.11,210.5,211.76,211.76,1422790,0.0000,1.0
2021-09-28,211.67,212.03,209.16,209.96,209.96,865865,0.0000,1.0


In [47]:
#check data types before performing any calculations
historical_prices.dtypes

1. open                 object
2. high                 object
3. low                  object
4. close                object
5. adjusted close       object
6. volume               object
7. dividend amount      object
8. split coefficient    object
dtype: object

In [48]:
#turn column values into numeric values
historical_prices = historical_prices.apply(pd.to_numeric)
historical_prices.dtypes


1. open                 float64
2. high                 float64
3. low                  float64
4. close                float64
5. adjusted close       float64
6. volume                 int64
7. dividend amount      float64
8. split coefficient    float64
dtype: object

In [49]:
#perform return calculations for each specific date
historical_prices['Intraday_Returns'] = round((historical_prices['5. adjusted close']/historical_prices['1. open']-1)*100,4)

#find the previous 5 day median intraday return and create the column 
#the value will take into account the previous 5 trading days
historical_prices['Weekly_Median_Return'] = historical_prices['Intraday_Returns'].rolling(window=5).median().shift(-5)

#display the first values
print(historical_prices.shape)

historical_prices.iloc[0:6,:]
#remember that the model could break if there is not enough historical data

(5517, 10)


,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return
2021-10-04,214.00,215.56,212.400,213.02,213.02,1196629,0.0,1.0,-0.4579,-0.3531
2021-10-01,211.59,216.14,210.940,213.48,213.48,1314205,0.0,1.0,0.8932,-0.5147
2021-09-30,214.89,215.79,210.460,210.69,210.69,1181815,0.0,1.0,-1.9545,-0.3531
2021-09-29,211.27,213.11,210.500,211.76,211.76,1422790,0.0,1.0,0.2319,-0.3531
2021-09-28,211.67,212.03,209.160,209.96,209.96,865865,0.0,1.0,-0.8079,-0.3531
2021-09-27,212.42,214.05,211.301,211.67,211.67,829371,0.0,1.0,-0.3531,-0.5147


In [50]:
#notice how the indexes are the dates?
#make that a column labeled reportedDate because that will be the key column for the inner join
historical_prices['reportedDate'] = historical_prices.index
#reset the index since its a column now
historical_prices.reset_index(drop=True, inplace=True)
historical_prices.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
0,214.00,215.56,212.40,213.02,213.02,1196629,0.0,1.0,-0.4579,-0.3531,2021-10-04
1,211.59,216.14,210.94,213.48,213.48,1314205,0.0,1.0,0.8932,-0.5147,2021-10-01
2,214.89,215.79,210.46,210.69,210.69,1181815,0.0,1.0,-1.9545,-0.3531,2021-09-30
3,211.27,213.11,210.50,211.76,211.76,1422790,0.0,1.0,0.2319,-0.3531,2021-09-29
4,211.67,212.03,209.16,209.96,209.96,865865,0.0,1.0,-0.8079,-0.3531,2021-09-28


In [51]:
#save a copy of this version of historical_prices
#this will be referenced in the future to get class variable 

master_hist = historical_prices.copy()
master_hist.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
0,214.00,215.56,212.40,213.02,213.02,1196629,0.0,1.0,-0.4579,-0.3531,2021-10-04
1,211.59,216.14,210.94,213.48,213.48,1314205,0.0,1.0,0.8932,-0.5147,2021-10-01
2,214.89,215.79,210.46,210.69,210.69,1181815,0.0,1.0,-1.9545,-0.3531,2021-09-30
3,211.27,213.11,210.50,211.76,211.76,1422790,0.0,1.0,0.2319,-0.3531,2021-09-29
4,211.67,212.03,209.16,209.96,209.96,865865,0.0,1.0,-0.8079,-0.3531,2021-09-28


In [52]:
#for training purposes,only keep the rows that are relevant.
#the api feeds in the most current earnings and as a result, the historical prices will not line up
#let those rows be used for the testing portion, while the remaining events get analyzed

#create a list of all the reportedDates in the earnings dataframe
reported_dates_list  = quarterly_earnings.reportedDate.tolist()


#remove the most recent reporting date
reported_dates_list = reported_dates_list[1:]

#tis line filters out all the dates that are not a previous reporting date
historical_prices = historical_prices.loc[historical_prices['reportedDate'].isin(reported_dates_list)]

historical_prices.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
124,224.54,226.765,220.98,224.21,222.731619,4390513,0.0,1.0,-0.8054,-1.1487,2021-04-08
186,233.00,240.760,227.31,228.87,226.617494,2537130,0.0,1.0,-2.7393,-0.6427,2021-01-07
253,191.00,191.000,183.41,184.61,182.048945,1899443,0.0,1.0,-4.6864,-0.9815,2020-10-01
317,182.93,189.920,182.04,185.88,182.502975,3462252,0.0,1.0,-0.2334,-1.7269,2020-07-01
378,135.01,135.010,125.61,132.17,129.177115,3276486,0.0,1.0,-4.3203,-0.7462,2020-04-03


In [53]:
#add the column that determines if the intraday return of the earnings release date is larger than that of the median
historical_prices['intraday_event'] = (historical_prices['Intraday_Returns'] > historical_prices['Weekly_Median_Return']).astype(int)

#add the column that determines if the adjusted share price is larger than that of last earnings call
historical_prices['adjusted_price_event'] = (historical_prices['5. adjusted close'] > historical_prices['5. adjusted close'].shift(-1)).astype(int)


#store the values of the adjusted close price in a list to be reference later for the class variable
adjusted_close_prices = historical_prices['5. adjusted close'].tolist()


#display new variables
historical_prices.head()


,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate,intraday_event,adjusted_price_event
124,224.54,226.765,220.98,224.21,222.731619,4390513,0.0,1.0,-0.8054,-1.1487,2021-04-08,1,0
186,233.00,240.760,227.31,228.87,226.617494,2537130,0.0,1.0,-2.7393,-0.6427,2021-01-07,0,1
253,191.00,191.000,183.41,184.61,182.048945,1899443,0.0,1.0,-4.6864,-0.9815,2020-10-01,0,0
317,182.93,189.920,182.04,185.88,182.502975,3462252,0.0,1.0,-0.2334,-1.7269,2020-07-01,1,1
378,135.01,135.010,125.61,132.17,129.177115,3276486,0.0,1.0,-4.3203,-0.7462,2020-04-03,0,0


In [54]:
#keep the relevant columns only
columns_to_keep = ['Intraday_Returns','Weekly_Median_Return','reportedDate', 'intraday_event','adjusted_price_event']
historical_prices = historical_prices[columns_to_keep]
historical_prices.head()

,Intraday_Returns,Weekly_Median_Return,reportedDate,intraday_event,adjusted_price_event
124,-0.8054,-1.1487,2021-04-08,1,0
186,-2.7393,-0.6427,2021-01-07,0,1
253,-4.6864,-0.9815,2020-10-01,0,0
317,-0.2334,-1.7269,2020-07-01,1,1
378,-4.3203,-0.7462,2020-04-03,0,0


In [55]:
#join the dataframes so that earnings calendar is on the left 
earn_hist_df = quarterly_earnings.iloc[1:,:].copy()


inner_merged_master = pd.merge(earn_hist_df, historical_prices)
inner_merged_master.head()


,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,Intraday_Returns,Weekly_Median_Return,intraday_event,adjusted_price_event
0,2021-02-28,2021-04-08,1.82,1.5484,0.2716,17.5407,-0.8054,-1.1487,1,0
1,2020-11-30,2021-01-07,3.09,2.4095,0.6805,28.2424,-2.7393,-0.6427,0,1
2,2020-08-31,2020-10-01,2.76,2.5132,0.2468,9.8201,-4.6864,-0.9815,0,0
3,2020-05-31,2020-07-01,2.3,2.0453,0.2547,12.4529,-0.2334,-1.7269,1,1
4,2020-02-29,2020-04-03,2.06,1.6572,0.4028,24.3061,-4.3203,-0.7462,0,0


In [56]:
#remove the fiscalDateEnding column
inner_merged_master = inner_merged_master.iloc[:,1:] 
inner_merged_master.head()

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,Intraday_Returns,Weekly_Median_Return,intraday_event,adjusted_price_event
0,2021-04-08,1.82,1.5484,0.2716,17.5407,-0.8054,-1.1487,1,0
1,2021-01-07,3.09,2.4095,0.6805,28.2424,-2.7393,-0.6427,0,1
2,2020-10-01,2.76,2.5132,0.2468,9.8201,-4.6864,-0.9815,0,0
3,2020-07-01,2.3,2.0453,0.2547,12.4529,-0.2334,-1.7269,1,1
4,2020-04-03,2.06,1.6572,0.4028,24.3061,-4.3203,-0.7462,0,0


In [57]:
#check for datatypes before doing calculations
inner_merged_master.dtypes


reportedDate             object
reportedEPS              object
estimatedEPS             object
surprise                 object
surprisePercentage       object
Intraday_Returns        float64
Weekly_Median_Return    float64
intraday_event            int32
adjusted_price_event      int32
dtype: object

In [58]:
#convert to numeric
inner_merged_master["reportedEPS"] = pd.to_numeric(inner_merged_master["reportedEPS"])

inner_merged_master["estimatedEPS"] = pd.to_numeric(inner_merged_master["estimatedEPS"])

inner_merged_master["surprisePercentage"] = pd.to_numeric(inner_merged_master["surprisePercentage"])

inner_merged_master.dtypes


reportedDate             object
reportedEPS             float64
estimatedEPS            float64
surprise                 object
surprisePercentage      float64
Intraday_Returns        float64
Weekly_Median_Return    float64
intraday_event            int32
adjusted_price_event      int32
dtype: object

In [59]:
#add the column that determines if current earnings were greater than last time
inner_merged_master['earnings_event'] = (inner_merged_master['reportedEPS'] > inner_merged_master['reportedEPS'].shift(-1)).astype(int)

# hist.loc[:,'DependentVariable'] = (hist['Close'] > hist['Close'].shift(-1)).astype(int)

#add the column that determines if the surprise variable is positive or not
inner_merged_master['surprise_event'] = (inner_merged_master['surprisePercentage'] > 0).astype(int)

#display new variables
inner_merged_master.head()

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,Intraday_Returns,Weekly_Median_Return,intraday_event,adjusted_price_event,earnings_event,surprise_event
0,2021-04-08,1.82,1.5484,0.2716,17.5407,-0.8054,-1.1487,1,0,0,1
1,2021-01-07,3.09,2.4095,0.6805,28.2424,-2.7393,-0.6427,0,1,1,1
2,2020-10-01,2.76,2.5132,0.2468,9.8201,-4.6864,-0.9815,0,0,1,1
3,2020-07-01,2.30,2.0453,0.2547,12.4529,-0.2334,-1.7269,1,1,1,1
4,2020-04-03,2.06,1.6572,0.4028,24.3061,-4.3203,-0.7462,0,0,0,1


In [60]:
#add the class variable to the inner_merged df
#do this by calling on the master_hist table
master_hist.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
0,214.00,215.56,212.40,213.02,213.02,1196629,0.0,1.0,-0.4579,-0.3531,2021-10-04
1,211.59,216.14,210.94,213.48,213.48,1314205,0.0,1.0,0.8932,-0.5147,2021-10-01
2,214.89,215.79,210.46,210.69,210.69,1181815,0.0,1.0,-1.9545,-0.3531,2021-09-30
3,211.27,213.11,210.50,211.76,211.76,1422790,0.0,1.0,0.2319,-0.3531,2021-09-29
4,211.67,212.03,209.16,209.96,209.96,865865,0.0,1.0,-0.8079,-0.3531,2021-09-28


In [61]:
#filter the table to show only reported dates
master_hist_1 = master_hist.loc[master_hist['reportedDate'].isin(reported_dates_list)]
master_hist_1.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
124,224.54,226.765,220.98,224.21,222.731619,4390513,0.0,1.0,-0.8054,-1.1487,2021-04-08
186,233.00,240.760,227.31,228.87,226.617494,2537130,0.0,1.0,-2.7393,-0.6427,2021-01-07
253,191.00,191.000,183.41,184.61,182.048945,1899443,0.0,1.0,-4.6864,-0.9815,2020-10-01
317,182.93,189.920,182.04,185.88,182.502975,3462252,0.0,1.0,-0.2334,-1.7269,2020-07-01
378,135.01,135.010,125.61,132.17,129.177115,3276486,0.0,1.0,-4.3203,-0.7462,2020-04-03


In [62]:
#store the index values from the list
row_index_list = master_hist_1.index.tolist()
row_index_list[0:5]

[124, 186, 253, 317, 378]

In [63]:
#alter the values of the elements in the list so that element = element -1
#this will give us the value of the index pertaining to returns one trading day following reported release
for i in range( len(row_index_list)):
    row_index_list[i] = row_index_list[i] - 1

row_index_list[0:5]

[123, 185, 252, 316, 377]

In [64]:
#view the historical price master df
master_hist.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
0,214.00,215.56,212.40,213.02,213.02,1196629,0.0,1.0,-0.4579,-0.3531,2021-10-04
1,211.59,216.14,210.94,213.48,213.48,1314205,0.0,1.0,0.8932,-0.5147,2021-10-01
2,214.89,215.79,210.46,210.69,210.69,1181815,0.0,1.0,-1.9545,-0.3531,2021-09-30
3,211.27,213.11,210.50,211.76,211.76,1422790,0.0,1.0,0.2319,-0.3531,2021-09-29
4,211.67,212.03,209.16,209.96,209.96,865865,0.0,1.0,-0.8079,-0.3531,2021-09-28


In [65]:
#lock the dataframe so that only the row indexes from the list come up
master_hist = master_hist.loc[row_index_list,:]
print(master_hist.shape)
master_hist.head()

(86, 11)


,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate
123,224.72,225.3200,218.6200,224.15,222.672015,2993298,0.0,1.0,-0.9113,-0.8054,2021-04-09
185,230.14,232.4700,225.2600,230.21,227.944306,1595373,0.0,1.0,-0.9541,-0.7279,2021-01-08
252,184.36,189.2962,183.2989,185.60,183.025210,1329040,0.0,1.0,-0.7240,-0.9815,2020-10-02
316,189.18,190.7400,185.3900,185.80,182.424429,1894401,0.0,1.0,-3.5710,-0.9078,2020-07-02
377,138.05,143.8100,137.0100,142.26,139.038635,2397357,0.0,1.0,0.7161,-3.1025,2020-04-06


In [66]:
#add a column that contains the adjusted close price of the reported date
master_hist['reported_adjusted_close'] = adjusted_close_prices
master_hist.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate,reported_adjusted_close
123,224.72,225.3200,218.6200,224.15,222.672015,2993298,0.0,1.0,-0.9113,-0.8054,2021-04-09,222.731619
185,230.14,232.4700,225.2600,230.21,227.944306,1595373,0.0,1.0,-0.9541,-0.7279,2021-01-08,226.617494
252,184.36,189.2962,183.2989,185.60,183.025210,1329040,0.0,1.0,-0.7240,-0.9815,2020-10-02,182.048945
316,189.18,190.7400,185.3900,185.80,182.424429,1894401,0.0,1.0,-3.5710,-0.9078,2020-07-02,182.502975
377,138.05,143.8100,137.0100,142.26,139.038635,2397357,0.0,1.0,0.7161,-3.1025,2020-04-06,129.177115


In [67]:
#calculate the adjusted price in a new column
master_hist['return_pct_difference'] = round((master_hist['5. adjusted close']/master_hist['reported_adjusted_close']-1)*100,2)
master_hist.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate,reported_adjusted_close,return_pct_difference
123,224.72,225.3200,218.6200,224.15,222.672015,2993298,0.0,1.0,-0.9113,-0.8054,2021-04-09,222.731619,-0.03
185,230.14,232.4700,225.2600,230.21,227.944306,1595373,0.0,1.0,-0.9541,-0.7279,2021-01-08,226.617494,0.59
252,184.36,189.2962,183.2989,185.60,183.025210,1329040,0.0,1.0,-0.7240,-0.9815,2020-10-02,182.048945,0.54
316,189.18,190.7400,185.3900,185.80,182.424429,1894401,0.0,1.0,-3.5710,-0.9078,2020-07-02,182.502975,-0.04
377,138.05,143.8100,137.0100,142.26,139.038635,2397357,0.0,1.0,0.7161,-3.1025,2020-04-06,129.177115,7.63


In [68]:
#calculate the class variable
master_hist['class_variable'] = (master_hist['return_pct_difference'] > 5).astype(int)
master_hist.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,Intraday_Returns,Weekly_Median_Return,reportedDate,reported_adjusted_close,return_pct_difference,class_variable
123,224.72,225.3200,218.6200,224.15,222.672015,2993298,0.0,1.0,-0.9113,-0.8054,2021-04-09,222.731619,-0.03,0
185,230.14,232.4700,225.2600,230.21,227.944306,1595373,0.0,1.0,-0.9541,-0.7279,2021-01-08,226.617494,0.59,0
252,184.36,189.2962,183.2989,185.60,183.025210,1329040,0.0,1.0,-0.7240,-0.9815,2020-10-02,182.048945,0.54,0
316,189.18,190.7400,185.3900,185.80,182.424429,1894401,0.0,1.0,-3.5710,-0.9078,2020-07-02,182.502975,-0.04,0
377,138.05,143.8100,137.0100,142.26,139.038635,2397357,0.0,1.0,0.7161,-3.1025,2020-04-06,129.177115,7.63,1


In [69]:
#take the class values from the master df and put them into the inner merged dataframe
class_values = master_hist['class_variable'].tolist()

#insert the values to the end of the dataframe
inner_merged_master['class_variable'] = class_values

inner_merged_master.head()

,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,Intraday_Returns,Weekly_Median_Return,intraday_event,adjusted_price_event,earnings_event,surprise_event,class_variable
0,2021-04-08,1.82,1.5484,0.2716,17.5407,-0.8054,-1.1487,1,0,0,1,0
1,2021-01-07,3.09,2.4095,0.6805,28.2424,-2.7393,-0.6427,0,1,1,1,0
2,2020-10-01,2.76,2.5132,0.2468,9.8201,-4.6864,-0.9815,0,0,1,1,0
3,2020-07-01,2.30,2.0453,0.2547,12.4529,-0.2334,-1.7269,1,1,1,1,0
4,2020-04-03,2.06,1.6572,0.4028,24.3061,-4.3203,-0.7462,0,0,0,1,1
